# Importing requirements

In [2]:
pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 4.5 MB/s eta 0:00:00

[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
import pandas as pd
import wfdb
import ast
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import classification_report
from collections import Counter
import time
from tqdm import tqdm


# Loading Data from dataset file

In [4]:
DATA_PATH = '/home/bmi-lab/Downloads/ptb-xl-a-large-publicly-available-electrocardiography-dataset-1.0.3'

ptbxl_df = pd.read_csv(os.path.join(DATA_PATH, 'ptbxl_database.csv'))
scp_statements = pd.read_csv(os.path.join(DATA_PATH, 'scp_statements.csv'), index_col=0)

diagnostic_scps = scp_statements[scp_statements['diagnostic'] == 1].index.values

scp_to_superclass = scp_statements['diagnostic_class'].to_dict()
scp_to_subclass = scp_statements['diagnostic_subclass'].to_dict()

In [5]:
ptbxl_df['scp_codes'] = ptbxl_df['scp_codes'].apply(lambda x: ast.literal_eval(x))

In [6]:
def aggregate_diagnostic_labels(df, scp_codes, scp_to_agg):
    df = df.copy()
    def aggregate_labels(scp_codes_dict):
        labels = set()
        for code in scp_codes_dict.keys():
            if code in scp_codes:
                label = scp_to_agg.get(code)
                if label:
                    labels.add(label)
        return list(labels)
    df['diagnostic_labels'] = df['scp_codes'].apply(aggregate_labels)
    return df

ptbxl_df = aggregate_diagnostic_labels(ptbxl_df, diagnostic_scps, scp_to_superclass)
ptbxl_df = ptbxl_df.rename(columns={'diagnostic_labels': 'superclass_labels'})

ptbxl_df = aggregate_diagnostic_labels(ptbxl_df, diagnostic_scps, scp_to_subclass)
ptbxl_df = ptbxl_df.rename(columns={'diagnostic_labels': 'subclass_labels'})

In [7]:
ptbxl_df = ptbxl_df[ptbxl_df['superclass_labels'].map(len) > 0]

In [8]:
train_df = ptbxl_df[ptbxl_df.strat_fold <= 8]
val_df = ptbxl_df[ptbxl_df.strat_fold == 9]
test_df = ptbxl_df[ptbxl_df.strat_fold == 10]

In [9]:
def load_data(df, sampling_rate, data_path):
    data = []
    if sampling_rate == 100:
        filenames = df['filename_lr'].values
    else:
        filenames = df['filename_hr'].values
    for filename in filenames:
        file_path = os.path.join(data_path, filename)
        signals, _ = wfdb.rdsamp(file_path)
        data.append(signals)
    return np.array(data)

X_train = load_data(train_df, sampling_rate=100, data_path=DATA_PATH)
X_val = load_data(val_df, sampling_rate=100, data_path=DATA_PATH)
X_test = load_data(test_df, sampling_rate=100, data_path=DATA_PATH)

In [10]:
train_labels_super = train_df['superclass_labels'].values
val_labels_super = val_df['superclass_labels'].values
test_labels_super = test_df['superclass_labels'].values

mlb_super = MultiLabelBinarizer()
y_train_super = mlb_super.fit_transform(train_labels_super)
y_val_super = mlb_super.transform(val_labels_super)
y_test_super = mlb_super.transform(test_labels_super)
classes_super = mlb_super.classes_

In [11]:
train_labels_sub = train_df['subclass_labels'].values
val_labels_sub = val_df['subclass_labels'].values
test_labels_sub = test_df['subclass_labels'].values

mlb_sub = MultiLabelBinarizer()
y_train_sub = mlb_sub.fit_transform(train_labels_sub)
y_val_sub = mlb_sub.transform(val_labels_sub)
y_test_sub = mlb_sub.transform(test_labels_sub)
classes_sub = mlb_sub.classes_

In [12]:
def normalize_data_per_channel(X):
    X = np.transpose(X, (0, 2, 1))
    mean = np.mean(X, axis=(0, 2), keepdims=True)
    std = np.std(X, axis=(0, 2), keepdims=True)
    X = (X - mean) / std
    X = np.transpose(X, (0, 2, 1))
    return X

X_train = normalize_data_per_channel(X_train)
X_val = normalize_data_per_channel(X_val)
X_test = normalize_data_per_channel(X_test)

In [13]:
class_counts_super = np.sum(y_train_super, axis=0)
total_samples_super = y_train_super.shape[0]

class_weight_super = {}
for i, count in enumerate(class_counts_super):
    class_weight_super[i] = total_samples_super / (len(class_counts_super) * count)

class_counts_sub = np.sum(y_train_sub, axis=0)
total_samples_sub = y_train_sub.shape[0]

class_weight_sub = {}
for i, count in enumerate(class_counts_sub):
    class_weight_sub[i] = total_samples_sub / (len(class_counts_sub) * count)

In [14]:
num_classes_super = y_train_super.shape[1]
class_totals = np.sum(y_train_super, axis=0)
class_weights = class_totals.max() / class_totals
weights_array = np.array(class_weights, dtype=np.float32)

In [15]:
num_classes_sub = y_train_sub.shape[1]
class_totals_sub = np.sum(y_train_sub, axis=0)
class_weights_sub = class_totals_sub.max() / class_totals_sub
weights_array_sub = np.array(class_weights_sub, dtype=np.float32)

In [16]:
y_train_super = y_train_super.astype(np.float32)
y_val_super = y_val_super.astype(np.float32)
y_test_super = y_test_super.astype(np.float32)

# Defining Entropy and Metrics

In [17]:
import tensorflow.keras.backend as K

def weighted_binary_crossentropy(weights):
    def loss(y_true, y_pred):
        weights_cast = K.cast(weights, y_pred.dtype)
        y_true = K.cast(y_true, y_pred.dtype)
        
        bce = K.binary_crossentropy(y_true, y_pred)
        weight_vector = y_true * weights_cast + (1 - y_true)
        weighted_bce = weight_vector * bce
        return K.mean(weighted_bce)
    return loss

def macro_f1(y_true, y_pred):
    y_true = K.cast(y_true, 'float32')
    y_pred = K.cast(y_pred, 'float32')
    y_pred = K.round(y_pred)
    
    tp = K.sum(y_true * y_pred, axis=0)
    fp = K.sum((1 - y_true) * y_pred, axis=0)
    fn = K.sum(y_true * (1 - y_pred), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    f1 = tf.where(tf.math.is_nan(f1), tf.zeros_like(f1), f1)
    return K.mean(f1)

# Defining Models

In [ ]:
def create_cnn_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)

    x = layers.Conv1D(64, kernel_size=7, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(128, kernel_size=5, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(256, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling1D(pool_size=2)(x)
    
    x = layers.Conv1D(512, kernel_size=3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.GlobalAveragePooling1D()(x)
    
    x = layers.Dense(1024, activation='relu')(x)
    x = layers.Dropout(0.1)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    
    model = models.Model(inputs, outputs)
    return model


In [19]:
# def create_resnet_model(input_shape, num_classes):
#     inputs = layers.Input(shape=input_shape)
#     x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation('relu')(x)
#     x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    
#     previous_filters = x.shape[-1]
#     for filters in [64, 128, 256]:
#         x_shortcut = x
#         strides = 1
#         if previous_filters != filters:
#             strides = 2

#         x = layers.Conv1D(filters, kernel_size=3, strides=strides, padding='same')(x)
#         x = layers.BatchNormalization()(x)
#         x = layers.Activation('relu')(x)
#         x = layers.Conv1D(filters, kernel_size=3, padding='same')(x)
#         x = layers.BatchNormalization()(x)
        
#         if previous_filters != filters or strides != 1:
#             x_shortcut = layers.Conv1D(filters, kernel_size=1, strides=strides, padding='same')(x_shortcut)
#             x_shortcut = layers.BatchNormalization()(x_shortcut)
        
#         x = layers.Add()([x, x_shortcut])
#         x = layers.Activation('relu')(x)
#         previous_filters = filters
#     x = layers.GlobalAveragePooling1D()(x)
#     outputs = layers.Dense(num_classes, activation='sigmoid')(x)
#     model = models.Model(inputs, outputs)
#     return model

In [20]:
def residual_block_1d(x, filters, kernel_size=3, strides=1, downsample=False):
    shortcut = x
    
    x = layers.Conv1D(filters, kernel_size=kernel_size, strides=strides, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)

    x = layers.Conv1D(filters, kernel_size=kernel_size, padding='same')(x)
    x = layers.BatchNormalization()(x)
    
    if downsample or shortcut.shape[-1] != filters:
        shortcut = layers.Conv1D(filters, kernel_size=1, strides=strides, padding='same')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.Activation('relu')(x)
    return x

def create_resnet_model(input_shape, num_classes):
    inputs = layers.Input(shape=input_shape)
    x = layers.Conv1D(64, kernel_size=7, strides=2, padding='same')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling1D(pool_size=3, strides=2, padding='same')(x)
    layers_filters = [64, 128, 256, 512]
    layers_blocks = [3, 4, 6, 3]

    for filters, num_blocks in zip(layers_filters, layers_blocks):
        for i in range(num_blocks):
            if i == 0 and filters != x.shape[-1]:
                x = residual_block_1d(x, filters, strides=2, downsample=True)
            else:
                x = residual_block_1d(x, filters)

    x = layers.GlobalAveragePooling1D()(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs, outputs)
    return model

In [21]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x

def create_vit_model(input_shape, num_classes):
    patch_size = 10 
    num_patches = input_shape[0] // patch_size
    projection_dim = 64
    num_heads = 4
    transformer_layers = 8
    mlp_head_units = [256, 128]
    dropout_rate = 0.1

    inputs = layers.Input(shape=input_shape)
    x = layers.Reshape((num_patches, patch_size * input_shape[1]))(inputs)
    x = layers.Dense(units=projection_dim)(x)
    positions = tf.range(start=0, limit=num_patches, delta=1)
    position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)
    x = x + position_embedding(positions)
    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(x)
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=dropout_rate
        )(x1, x1)
        x2 = layers.Add()([attention_output, x])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = mlp(x3, hidden_units=[projection_dim * 2, projection_dim], dropout_rate=dropout_rate)
        x = layers.Add()([x3, x2])
    x = layers.LayerNormalization(epsilon=1e-6)(x)
    x = layers.Flatten()(x)
    x = layers.Dropout(dropout_rate)(x)
    outputs = layers.Dense(num_classes, activation='sigmoid')(x)
    model = models.Model(inputs=inputs, outputs=outputs)
    return model

# Defining the training loop

In [22]:
def train_model(model, X_train, y_train, X_val, y_val, class_weight, batch_size=64, epochs=25):
    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy', macro_f1]
    )
    callbacks = [
        tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5),
        tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    ]
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        class_weight=class_weight
    )
    return history

# Training and Evaluating Models without CL

In [23]:
input_shape = X_train.shape[1:]
num_classes_super = y_train_super.shape[1]

cnn_super_model = create_cnn_model(input_shape, num_classes_super)
train_model(cnn_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

2024-11-19 16:09:31.270097: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.549533: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.552110: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-11-19 16:09:31.555656: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA
To enable them in other 

Epoch 1/25


2024-11-19 16:09:33.322857: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8907
2024-11-19 16:09:34.093169: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2024-11-19 16:09:34.191615: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x73d9f8042e20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-11-19 16:09:34.191677: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 4080, Compute Capability 8.9
2024-11-19 16:09:34.242857: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-19 16:09:34.547318: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of 

267/267 [==============================] - 5s 7ms/step - loss: 0.2797 - accuracy: 0.6384 - macro_f1: 0.6584 - val_loss: 0.3618 - val_accuracy: 0.6342 - val_macro_f1: 0.6274 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2422 - accuracy: 0.6853 - macro_f1: 0.7098 - val_loss: 0.3546 - val_accuracy: 0.6696 - val_macro_f1: 0.6617 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2281 - accuracy: 0.7005 - macro_f1: 0.7294 - val_loss: 0.3061 - val_accuracy: 0.6873 - val_macro_f1: 0.7062 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 7ms/step - loss: 0.2208 - accuracy: 0.7081 - macro_f1: 0.7385 - val_loss: 0.3125 - val_accuracy: 0.6752 - val_macro_f1: 0.6934 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 6ms/step - loss: 0.2149 - accuracy: 0.7142 - macro_f1: 0.7469 - val_loss: 0.3087 - val_accuracy: 0.6887 - val_macro_f1: 0.7001 - lr: 0.0010
Epoch 6/25
267/267 [======

In [24]:
resnet_super_model = create_resnet_model(input_shape, num_classes_super)
train_model(resnet_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

Epoch 1/25
267/267 [==============================] - 14s 22ms/step - loss: 0.3015 - accuracy: 0.6271 - macro_f1: 0.6294 - val_loss: 0.6704 - val_accuracy: 0.3882 - val_macro_f1: 0.4309 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2545 - accuracy: 0.6802 - macro_f1: 0.6948 - val_loss: 0.3448 - val_accuracy: 0.6561 - val_macro_f1: 0.6679 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2375 - accuracy: 0.6924 - macro_f1: 0.7167 - val_loss: 0.4898 - val_accuracy: 0.5415 - val_macro_f1: 0.6120 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2312 - accuracy: 0.6977 - macro_f1: 0.7230 - val_loss: 0.3757 - val_accuracy: 0.6789 - val_macro_f1: 0.6919 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2215 - accuracy: 0.7141 - macro_f1: 0.7379 - val_loss: 0.3218 - val_accuracy: 0.6664 - val_macro_f1: 0.6708 - lr: 0.0010
Epoch 6/2

In [25]:
vit_super_model = create_vit_model(input_shape, num_classes_super)
train_model(vit_super_model, X_train, y_train_super, X_val, y_val_super, class_weight_super)

Epoch 1/25
267/267 [==============================] - 13s 20ms/step - loss: 0.3824 - accuracy: 0.4987 - macro_f1: 0.4738 - val_loss: 0.3772 - val_accuracy: 0.5718 - val_macro_f1: 0.5947 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2926 - accuracy: 0.6282 - macro_f1: 0.6302 - val_loss: 0.3544 - val_accuracy: 0.6398 - val_macro_f1: 0.5986 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2587 - accuracy: 0.6657 - macro_f1: 0.6803 - val_loss: 0.3343 - val_accuracy: 0.6654 - val_macro_f1: 0.6820 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.2377 - accuracy: 0.6905 - macro_f1: 0.7151 - val_loss: 0.3849 - val_accuracy: 0.6454 - val_macro_f1: 0.6654 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 20ms/step - loss: 0.2215 - accuracy: 0.7009 - macro_f1: 0.7318 - val_loss: 0.3230 - val_accuracy: 0.6827 - val_macro_f1: 0.6708 - lr: 0.0010
Epoch 6/2

In [26]:
num_classes_sub = y_train_sub.shape[1]
cnn_sub_model = create_cnn_model(input_shape, num_classes_sub)
train_model(cnn_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 3s 7ms/step - loss: 0.1084 - accuracy: 0.3936 - macro_f1: 0.1386 - val_loss: 0.1403 - val_accuracy: 0.5303 - val_macro_f1: 0.1263 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0899 - accuracy: 0.4710 - macro_f1: 0.1888 - val_loss: 0.1414 - val_accuracy: 0.5065 - val_macro_f1: 0.1767 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0840 - accuracy: 0.4974 - macro_f1: 0.2213 - val_loss: 0.1536 - val_accuracy: 0.4623 - val_macro_f1: 0.1564 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0775 - accuracy: 0.5103 - macro_f1: 0.2367 - val_loss: 0.1536 - val_accuracy: 0.3975 - val_macro_f1: 0.2138 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 7ms/step - loss: 0.0703 - accuracy: 0.5210 - macro_f1: 0.2643 - val_loss: 0.1270 - val_accuracy: 0.5163 - val_macro_f1: 0.2207 - lr: 0.0010
Epoch 6/25
267/

In [27]:
resnet_sub_model = create_resnet_model(input_shape, num_classes_sub)
train_model(resnet_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 14s 21ms/step - loss: 0.1181 - accuracy: 0.3439 - macro_f1: 0.0673 - val_loss: 0.1689 - val_accuracy: 0.4264 - val_macro_f1: 0.1547 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 20ms/step - loss: 0.1026 - accuracy: 0.4255 - macro_f1: 0.1100 - val_loss: 0.3509 - val_accuracy: 0.0974 - val_macro_f1: 0.0624 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0921 - accuracy: 0.4473 - macro_f1: 0.1355 - val_loss: 0.1577 - val_accuracy: 0.4567 - val_macro_f1: 0.1011 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0856 - accuracy: 0.4694 - macro_f1: 0.1732 - val_loss: 0.1486 - val_accuracy: 0.5140 - val_macro_f1: 0.1834 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0831 - accuracy: 0.4777 - macro_f1: 0.1829 - val_loss: 0.1678 - val_accuracy: 0.3551 - val_macro_f1: 0.1285 - lr: 0.0010
Epoch 6/2

In [28]:
vit_sub_model = create_vit_model(input_shape, num_classes_sub)
train_model(vit_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 13s 20ms/step - loss: 0.1335 - accuracy: 0.2333 - macro_f1: 0.0649 - val_loss: 0.1869 - val_accuracy: 0.2773 - val_macro_f1: 0.0707 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0894 - accuracy: 0.4011 - macro_f1: 0.1432 - val_loss: 0.1557 - val_accuracy: 0.4357 - val_macro_f1: 0.1488 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0701 - accuracy: 0.4798 - macro_f1: 0.2268 - val_loss: 0.1522 - val_accuracy: 0.4543 - val_macro_f1: 0.1872 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0587 - accuracy: 0.5259 - macro_f1: 0.2836 - val_loss: 0.1545 - val_accuracy: 0.4278 - val_macro_f1: 0.1870 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0489 - accuracy: 0.5568 - macro_f1: 0.3374 - val_loss: 0.1516 - val_accuracy: 0.5037 - val_macro_f1: 0.2110 - lr: 0.0010
Epoch 6/2

In [29]:
def evaluate_model(model, X_test, y_test, classes):
    y_pred = model.predict(X_test)
    y_pred_threshold = (y_pred >= 0.5).astype(int)
    report = classification_report(y_test, y_pred_threshold, target_names=classes, zero_division=0, output_dict=True)
    print(classification_report(y_test, y_pred_threshold, target_names=classes, zero_division=0))
    return report


In [30]:
print("CNN Superdiagnostic Classification Report:")
cnn_super_report = evaluate_model(cnn_super_model, X_test, y_test_super, classes_super)

print("ResNet Superdiagnostic Classification Report:")
resnet_super_report = evaluate_model(resnet_super_model, X_test, y_test_super, classes_super)

print("ViT Superdiagnostic Classification Report:")
vit_super_report = evaluate_model(vit_super_model, X_test, y_test_super, classes_super)


CNN Superdiagnostic Classification Report:
68/68 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

          CD       0.84      0.68      0.75       496
         HYP       0.56      0.65      0.60       262
          MI       0.84      0.61      0.71       550
        NORM       0.83      0.88      0.86       963
        STTC       0.80      0.71      0.75       521

   micro avg       0.80      0.74      0.77      2792
   macro avg       0.77      0.71      0.73      2792
weighted avg       0.80      0.74      0.76      2792
 samples avg       0.77      0.76      0.75      2792

ResNet Superdiagnostic Classification Report:
68/68 [==============================] - 1s 4ms/step
              precision    recall  f1-score   support

          CD       0.81      0.69      0.75       496
         HYP       0.72      0.50      0.59       262
          MI       0.72      0.77      0.74       550
        NORM       0.87      0.82      0.84  

In [31]:
print("CNN Subdiagnostic Classification Report:")
cnn_sub_report = evaluate_model(cnn_sub_model, X_test, y_test_sub, classes_sub)

print("ResNet Subdiagnostic Classification Report:")
resnet_sub_report = evaluate_model(resnet_sub_model, X_test, y_test_sub, classes_sub)

print("ViT Subdiagnostic Classification Report:")
vit_sub_report = evaluate_model(vit_sub_model, X_test, y_test_sub, classes_sub)


CNN Subdiagnostic Classification Report:
68/68 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         AMI       0.88      0.53      0.66       306
       CLBBB       0.91      0.91      0.91        54
       CRBBB       0.79      0.91      0.84        54
       ILBBB       0.12      0.12      0.12         8
         IMI       0.79      0.48      0.60       327
       IRBBB       0.56      0.68      0.62       112
        ISCA       0.54      0.16      0.25        93
        ISCI       0.44      0.30      0.36        40
        ISC_       0.72      0.47      0.57       128
        IVCD       0.19      0.06      0.09        79
   LAFB/LPFB       0.80      0.64      0.71       179
     LAO/LAE       0.00      0.00      0.00        42
         LMI       0.30      0.15      0.20        20
         LVH       0.75      0.53      0.62       214
        NORM       0.88      0.76      0.81       963
        NST_       0.32      0.16      0.

# Defining and Training on LwF

In [32]:
cnn_soft_targets_super = cnn_super_model.predict(X_train)

def lwf_loss(y_true, y_pred, old_predictions, T=2):
    task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    dist_loss = tf.keras.losses.KLDivergence()(tf.nn.softmax(old_predictions / T),
                                               tf.nn.softmax(y_pred / T))
    total_loss = task_loss + dist_loss
    return total_loss

print("Working on CNN for LwF Now:")
cnn_model_lwf = create_cnn_model(input_shape, num_classes_sub)
cnn_model_lwf.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: lwf_loss(y_true, y_pred, old_predictions=cnn_soft_targets_super),
    metrics=[macro_f1]
)
train_model(cnn_model_lwf, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

print("Working on ResNet for LwF Now:")
resnet_soft_targets_super = resnet_super_model.predict(X_train)
resnet_model_lwf = create_resnet_model(input_shape, num_classes_sub)
resnet_model_lwf.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: lwf_loss(y_true, y_pred, old_predictions=resnet_soft_targets_super),
    metrics=[macro_f1]
)
train_model(resnet_model_lwf, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

print("Working on ViT for LwF Now:")
vit_soft_targets_super = vit_super_model.predict(X_train)
vit_model_lwf = create_vit_model(input_shape, num_classes_sub)
vit_model_lwf.compile(
    optimizer='adam',
    loss=lambda y_true, y_pred: lwf_loss(y_true, y_pred, old_predictions=vit_soft_targets_super),
    metrics=[macro_f1]
)
train_model(vit_model_lwf, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)


534/534 [==============================] - 1s 1ms/step
Working on CNN for LwF Now:
Epoch 1/25
267/267 [==============================] - 3s 7ms/step - loss: 0.1081 - accuracy: 0.4157 - macro_f1: 0.1335 - val_loss: 0.1729 - val_accuracy: 0.3565 - val_macro_f1: 0.1212 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0866 - accuracy: 0.4786 - macro_f1: 0.1963 - val_loss: 0.1272 - val_accuracy: 0.5536 - val_macro_f1: 0.1921 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0817 - accuracy: 0.4922 - macro_f1: 0.2250 - val_loss: 0.1319 - val_accuracy: 0.5238 - val_macro_f1: 0.2143 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0743 - accuracy: 0.5280 - macro_f1: 0.2534 - val_loss: 0.1643 - val_accuracy: 0.3723 - val_macro_f1: 0.1882 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0695 - accuracy: 0.5175 - macro_f1: 0.2734 - val_loss

# Defining and Training on EwC

In [33]:
class EWC:
    def __init__(self, model, X, y, batch_size=32, exclude_params=[]):
        self.model = model
        self.params = {}
        for p in model.trainable_variables:
            if id(p) not in exclude_params:
                self.params[id(p)] = p.numpy()
        self.fisher = self.compute_fisher(X, y, batch_size, exclude_params)

    def compute_fisher(self, X, y, batch_size, exclude_params):
        fisher = {}
        num_samples = X.shape[0]
        num_batches = int(np.ceil(num_samples / batch_size))

        for batch_idx in range(num_batches):
            X_batch = X[batch_idx*batch_size:(batch_idx+1)*batch_size]
            y_batch = y[batch_idx*batch_size:(batch_idx+1)*batch_size]
            with tf.GradientTape() as tape:
                preds = self.model(X_batch)
                loss = tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_batch, preds))
            grads = tape.gradient(loss, self.model.trainable_variables)
            for p, g in zip(self.model.trainable_variables, grads):
                if g is not None and id(p) not in exclude_params:
                    param_id = id(p)
                    if param_id not in fisher:
                        fisher[param_id] = np.square(g.numpy())
                    else:
                        fisher[param_id] += np.square(g.numpy())
        for k in fisher.keys():
            fisher[k] /= num_batches
        return fisher

    def penalty(self, model):
        loss = 0
        for p in model.trainable_variables:
            param_id = id(p)
            if param_id in self.fisher:
                fisher = tf.convert_to_tensor(self.fisher[param_id])
                loss += tf.reduce_sum(fisher * tf.square(p - self.params[param_id]))
        return loss

In [34]:
def modify_model_for_subdiagnostic(base_model, num_classes_sub):
    x = base_model.layers[-2].output
    outputs = layers.Dense(num_classes_sub, activation='sigmoid', name='output_sub')(x)
    new_model = models.Model(inputs=base_model.input, outputs=outputs)
    return new_model


In [35]:
lambda_ewc = 1000
cnn_sub_model = modify_model_for_subdiagnostic(cnn_super_model, num_classes_sub)
exclude_params_cnn = [id(w) for w in cnn_sub_model.layers[-1].trainable_weights]
ewc_cnn = EWC(cnn_super_model, X_train, y_train_super, exclude_params=exclude_params_cnn)

def ewc_loss_cnn(y_true, y_pred):
    task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    ewc_penalty = ewc_cnn.penalty(cnn_sub_model)
    total_loss = task_loss + (lambda_ewc / 2) * ewc_penalty
    return total_loss

cnn_sub_model.compile(
    optimizer='adam',
    loss=ewc_loss_cnn,
    metrics=[macro_f1]
)

train_model(cnn_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)


Epoch 1/25
267/267 [==============================] - 3s 7ms/step - loss: 0.0898 - accuracy: 0.5516 - macro_f1: 0.2330 - val_loss: 0.1200 - val_accuracy: 0.5727 - val_macro_f1: 0.2476 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0689 - accuracy: 0.5959 - macro_f1: 0.2996 - val_loss: 0.1143 - val_accuracy: 0.5708 - val_macro_f1: 0.2700 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0612 - accuracy: 0.5940 - macro_f1: 0.3263 - val_loss: 0.1408 - val_accuracy: 0.4478 - val_macro_f1: 0.2596 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 2s 6ms/step - loss: 0.0589 - accuracy: 0.6023 - macro_f1: 0.3273 - val_loss: 0.1155 - val_accuracy: 0.5727 - val_macro_f1: 0.2822 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 2s 7ms/step - loss: 0.0562 - accuracy: 0.6028 - macro_f1: 0.3462 - val_loss: 0.1235 - val_accuracy: 0.5634 - val_macro_f1: 0.2811 - lr: 0.0010
Epoch 6/25
267/

In [36]:
resnet_sub_model = modify_model_for_subdiagnostic(resnet_super_model, num_classes_sub)
exclude_params_resnet = [id(w) for w in resnet_sub_model.layers[-1].trainable_weights]
ewc_resnet = EWC(resnet_super_model, X_train, y_train_super, exclude_params=exclude_params_resnet)
def ewc_loss_resnet(y_true, y_pred):
    task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    ewc_penalty = ewc_resnet.penalty(resnet_sub_model)
    total_loss = task_loss + (lambda_ewc / 2) * ewc_penalty
    return total_loss

resnet_sub_model.compile(
    optimizer='adam',
    loss=ewc_loss_resnet,
    metrics=[macro_f1]
)

train_model(resnet_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)


Epoch 1/25
267/267 [==============================] - 14s 21ms/step - loss: 0.0919 - accuracy: 0.5074 - macro_f1: 0.1920 - val_loss: 12.8419 - val_accuracy: 0.0140 - val_macro_f1: 0.0163 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0824 - accuracy: 0.5193 - macro_f1: 0.2222 - val_loss: 0.1560 - val_accuracy: 0.3607 - val_macro_f1: 0.1969 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0730 - accuracy: 0.5496 - macro_f1: 0.2536 - val_loss: 0.1488 - val_accuracy: 0.4087 - val_macro_f1: 0.1990 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0707 - accuracy: 0.5542 - macro_f1: 0.2728 - val_loss: 0.1373 - val_accuracy: 0.4637 - val_macro_f1: 0.2351 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0618 - accuracy: 0.5713 - macro_f1: 0.3038 - val_loss: 0.1622 - val_accuracy: 0.3672 - val_macro_f1: 0.2348 - lr: 0.0010
Epoch 6/

In [37]:
vit_sub_model = modify_model_for_subdiagnostic(vit_super_model, num_classes_sub)
exclude_params_vit = [id(w) for w in vit_sub_model.layers[-1].trainable_weights]
ewc_vit = EWC(vit_super_model, X_train, y_train_super, exclude_params=exclude_params_vit)

def ewc_loss_vit(y_true, y_pred):
    task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    ewc_penalty = ewc_vit.penalty(vit_sub_model)
    total_loss = task_loss + (lambda_ewc / 2) * ewc_penalty
    return total_loss

vit_sub_model.compile(
    optimizer='adam',
    loss=ewc_loss_vit,
    metrics=[macro_f1]
)

train_model(vit_sub_model, X_train, y_train_sub, X_val, y_val_sub, class_weight_sub)

Epoch 1/25
267/267 [==============================] - 13s 21ms/step - loss: 0.0994 - accuracy: 0.4673 - macro_f1: 0.1712 - val_loss: 0.1509 - val_accuracy: 0.4897 - val_macro_f1: 0.1730 - lr: 0.0010
Epoch 2/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0730 - accuracy: 0.5453 - macro_f1: 0.2432 - val_loss: 0.1399 - val_accuracy: 0.4902 - val_macro_f1: 0.2325 - lr: 0.0010
Epoch 3/25
267/267 [==============================] - 5s 18ms/step - loss: 0.0555 - accuracy: 0.5749 - macro_f1: 0.3039 - val_loss: 0.1271 - val_accuracy: 0.5792 - val_macro_f1: 0.2226 - lr: 0.0010
Epoch 4/25
267/267 [==============================] - 5s 20ms/step - loss: 0.0454 - accuracy: 0.6127 - macro_f1: 0.3597 - val_loss: 0.1279 - val_accuracy: 0.5760 - val_macro_f1: 0.2620 - lr: 0.0010
Epoch 5/25
267/267 [==============================] - 5s 19ms/step - loss: 0.0405 - accuracy: 0.6345 - macro_f1: 0.3964 - val_loss: 0.1404 - val_accuracy: 0.5555 - val_macro_f1: 0.2264 - lr: 0.0010
Epoch 6/2

# Defining and Training on SI

In [ ]:
class SI:
    def __init__(self, prev_model, damping_factor=0.1, exclude_params=[]):
        self.prev_params = {}
        self.omega = {}
        self.damping_factor = damping_factor
        self.exclude_params = exclude_params
        
        for var in prev_model.trainable_variables:
            if var.name not in self.exclude_params:
                self.prev_params[var.name] = var.numpy()
                self.omega[var.name] = np.zeros_like(var.numpy())

    def accumulate_importance(self, model, grads):
        for var, grad in zip(model.trainable_variables, grads):
            if grad is not None and var.name in self.omega:
                delta_theta = var.numpy() - self.prev_params[var.name]
                self.omega[var.name] += grad.numpy() * delta_theta

    def update_omega(self, model):
        for var in model.trainable_variables:
            if var.name in self.omega:
                delta_theta = var.numpy() - self.prev_params[var.name]
                denom = np.square(delta_theta) + self.damping_factor
                # Avoid division by zero
                denom = np.where(denom == 0, self.damping_factor, denom)
                self.omega[var.name] = self.omega[var.name] / denom

    def penalty(self, model):
        loss = 0
        for var in model.trainable_variables:
            if var.name in self.omega:
                omega = tf.convert_to_tensor(self.omega[var.name], dtype=var.dtype)
                prev_param = tf.convert_to_tensor(self.prev_params[var.name], dtype=var.dtype)
                loss += tf.reduce_sum(omega * tf.square(var - prev_param))
        return loss


In [ ]:
num_classes_sub = y_train_sub.shape[1]
cnn_sub_model = modify_model_for_subdiagnostic(cnn_super_model, num_classes_sub)

exclude_params_cnn = [w.name for w in cnn_sub_model.layers[-1].trainable_weights]
si_cnn = SI(cnn_sub_model, exclude_params=exclude_params_cnn)


In [40]:
lambda_si = 1.0
epochs = 25
batch_size = 64
optimizer = tf.keras.optimizers.Adam()

train_macro_f1 = tf.keras.metrics.Mean(name='train_macro_f1')
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_macro_f1 = tf.keras.metrics.Mean(name='val_macro_f1')
val_loss = tf.keras.metrics.Mean(name='val_loss')

for epoch in range(epochs):
    start_time = time.time()
    print(f'\nCNN Epoch {epoch+1}/{epochs}')
    train_macro_f1.reset_state()
    train_loss.reset_state()
    
    for step in tqdm(range(len(X_train) // batch_size)):
        X_batch = X_train[step*batch_size:(step+1)*batch_size]
        y_batch = y_train_sub[step*batch_size:(step+1)*batch_size]
        
        with tf.GradientTape() as tape:
            preds = cnn_sub_model(X_batch, training=True)
            task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
            si_penalty = si_cnn.penalty(cnn_sub_model)
            total_loss = tf.reduce_mean(task_loss + lambda_si * si_penalty)
        
        grads = tape.gradient(total_loss, cnn_sub_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, cnn_sub_model.trainable_variables))
        si_cnn.accumulate_importance(cnn_sub_model, grads)
        
        batch_macro_f1 = macro_f1(y_batch, preds)
        train_macro_f1.update_state(batch_macro_f1)
        train_loss.update_state(total_loss)
        
    si_cnn.update_omega(cnn_sub_model)
    epoch_time = time.time() - start_time
    
    val_macro_f1.reset_state()
    val_loss.reset_state()
    for step in range(len(X_val) // batch_size):
        X_batch = X_val[step*batch_size:(step+1)*batch_size]
        y_batch = y_val_sub[step*batch_size:(step+1)*batch_size]
        preds = cnn_sub_model(X_batch, training=False)
        task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
        total_loss = tf.reduce_mean(task_loss)

        batch_macro_f1 = macro_f1(y_batch, preds)
        val_macro_f1.update_state(batch_macro_f1)
        val_loss.update_state(total_loss)
    print(f'Epoch {epoch+1}/{epochs}, '
          f'Time: {epoch_time:.2f}s, '
          f'Loss: {train_loss.result():.4f}, '
          f'Macro F1: {train_macro_f1.result():.4f}, '
          f'Val Loss: {val_loss.result():.4f}, '
          f'Val Macro F1: {val_macro_f1.result():.4f}')



CNN Epoch 1/25


  0%|          | 0/266 [00:00<?, ?it/s]

100%|██████████| 266/266 [00:07<00:00, 33.86it/s]


Epoch 1/25, Time: 7.86s, Loss: 0.1020, Macro F1: 0.3096, Val Loss: 0.0982, Val Macro F1: 0.3241

CNN Epoch 2/25


100%|██████████| 266/266 [00:07<00:00, 37.15it/s]


...

Epoch 23/25, Time: 7.08s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

CNN Epoch 24/25


100%|██████████| 266/266 [00:07<00:00, 37.52it/s]


Epoch 24/25, Time: 7.10s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

CNN Epoch 25/25


100%|██████████| 266/266 [00:07<00:00, 37.55it/s]


Epoch 25/25, Time: 7.09s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000


In [ ]:
resnet_sub_model = modify_model_for_subdiagnostic(resnet_super_model, num_classes_sub)
exclude_params_resnet = [w.name for w in resnet_sub_model.layers[-1].trainable_weights]
si_resnet = SI(resnet_sub_model, exclude_params=exclude_params_resnet)

In [42]:
lambda_si = 1.0  
epochs = 25
batch_size = 64
optimizer = tf.keras.optimizers.Adam()

train_macro_f1 = tf.keras.metrics.Mean(name='train_macro_f1')
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_macro_f1 = tf.keras.metrics.Mean(name='val_macro_f1')
val_loss = tf.keras.metrics.Mean(name='val_loss')

for epoch in range(epochs):
    start_time = time.time()
    print(f'\nResNet Epoch {epoch+1}/{epochs}')
    train_macro_f1.reset_state()
    train_loss.reset_state()
    
    for step in tqdm(range(len(X_train) // batch_size)):
        X_batch = X_train[step*batch_size:(step+1)*batch_size]
        y_batch = y_train_sub[step*batch_size:(step+1)*batch_size]
        
        with tf.GradientTape() as tape:
            preds = resnet_sub_model(X_batch, training=True)
            task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
            si_penalty = si_resnet.penalty(resnet_sub_model)
            total_loss = tf.reduce_mean(task_loss + lambda_si * si_penalty)
        
        grads = tape.gradient(total_loss, resnet_sub_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, resnet_sub_model.trainable_variables))
        si_resnet.accumulate_importance(resnet_sub_model, grads)
    
        batch_macro_f1 = macro_f1(y_batch, preds)
        train_macro_f1.update_state(batch_macro_f1)
        train_loss.update_state(total_loss)
    
    si_resnet.update_omega(resnet_sub_model)
    epoch_time = time.time() - start_time

    val_macro_f1.reset_state()
    val_loss.reset_state()
    for step in range(len(X_val) // batch_size):
        X_batch = X_val[step*batch_size:(step+1)*batch_size]
        y_batch = y_val_sub[step*batch_size:(step+1)*batch_size]
        preds = resnet_sub_model(X_batch, training=False)
        task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
        total_loss = tf.reduce_mean(task_loss)

        batch_macro_f1 = macro_f1(y_batch, preds)
        val_macro_f1.update_state(batch_macro_f1)
        val_loss.update_state(total_loss)

    print(f'Epoch {epoch+1}/{epochs}, '
          f'Time: {epoch_time:.2f}s, '
          f'Loss: {train_loss.result():.4f}, '
          f'Macro F1: {train_macro_f1.result():.4f}, '
          f'Val Loss: {val_loss.result():.4f}, '
          f'Val Macro F1: {val_macro_f1.result():.4f}')



ResNet Epoch 1/25


100%|██████████| 266/266 [00:49<00:00,  5.37it/s]


Epoch 1/25, Time: 49.55s, Loss: 0.0876, Macro F1: 0.2896, Val Loss: 0.1146, Val Macro F1: 0.2826

ResNet Epoch 2/25


100%|██████████| 266/266 [00:45<00:00,  5.87it/s]


Epoch 2/25, Time: 45.33s, Loss: nan, Macro F1: 0.1216, Val Loss: nan, Val Macro F1: 0.0000

ResNet Epoch 3/25


...

Epoch 23/25, Time: 45.27s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

ResNet Epoch 24/25


100%|██████████| 266/266 [00:44<00:00,  5.95it/s]


Epoch 24/25, Time: 44.76s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

ResNet Epoch 25/25


100%|██████████| 266/266 [00:45<00:00,  5.85it/s]


Epoch 25/25, Time: 45.47s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000


In [ ]:
vit_sub_model = modify_model_for_subdiagnostic(vit_super_model, num_classes_sub)
exclude_params_vit = [w.name for w in vit_sub_model.layers[-1].trainable_weights]
si_vit = SI(vit_sub_model, exclude_params=exclude_params_vit)


In [44]:
lambda_si = 1.0
epochs = 25
batch_size = 64
optimizer = tf.keras.optimizers.Adam()

train_macro_f1 = tf.keras.metrics.Mean(name='train_macro_f1')
train_loss = tf.keras.metrics.Mean(name='train_loss')
val_macro_f1 = tf.keras.metrics.Mean(name='val_macro_f1')
val_loss = tf.keras.metrics.Mean(name='val_loss')

for epoch in range(epochs):
    start_time = time.time()
    print(f'\nViT Epoch {epoch+1}/{epochs}')
    
    train_macro_f1.reset_state()
    train_loss.reset_state()

    for step in tqdm(range(len(X_train) // batch_size)):
        X_batch = X_train[step*batch_size:(step+1)*batch_size]
        y_batch = y_train_sub[step*batch_size:(step+1)*batch_size]
        
        with tf.GradientTape() as tape:
            preds = vit_sub_model(X_batch, training=True)
            task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
            si_penalty = si_vit.penalty(vit_sub_model)
            total_loss = tf.reduce_mean(task_loss + lambda_si * si_penalty)
        
        grads = tape.gradient(total_loss, vit_sub_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, vit_sub_model.trainable_variables))
        
        si_vit.accumulate_importance(vit_sub_model, grads)
        batch_macro_f1 = macro_f1(y_batch, preds)
        train_macro_f1.update_state(batch_macro_f1)
        train_loss.update_state(total_loss)
        
    si_vit.update_omega(vit_sub_model)
    epoch_time = time.time() - start_time
    val_macro_f1.reset_state()
    val_loss.reset_state()
    for step in range(len(X_val) // batch_size):
        X_batch = X_val[step*batch_size:(step+1)*batch_size]
        y_batch = y_val_sub[step*batch_size:(step+1)*batch_size]
        preds = vit_sub_model(X_batch, training=False)
        task_loss = tf.keras.losses.binary_crossentropy(y_batch, preds)
        total_loss = tf.reduce_mean(task_loss)
        
        batch_macro_f1 = macro_f1(y_batch, preds)
        val_macro_f1.update_state(batch_macro_f1)
        val_loss.update_state(total_loss)
        
    print(f'Epoch {epoch+1}/{epochs}, '
          f'Time: {epoch_time:.2f}s, '
          f'Loss: {train_loss.result():.4f}, '
          f'Macro F1: {train_macro_f1.result():.4f}, '
          f'Val Loss: {val_loss.result():.4f}, '
          f'Val Macro F1: {val_macro_f1.result():.4f}')



ViT Epoch 1/25


100%|██████████| 266/266 [00:40<00:00,  6.59it/s]


Epoch 1/25, Time: 40.37s, Loss: 0.1119, Macro F1: 0.2410, Val Loss: 0.1109, Val Macro F1: 0.2454

ViT Epoch 2/25


100%|██████████| 266/266 [00:35<00:00,  7.57it/s]


Epoch 2/25, Time: 35.15s, Loss: nan, Macro F1: 0.1772, Val Loss: nan, Val Macro F1: 0.0000

ViT Epoch 3/25


...

Epoch 23/25, Time: 35.92s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

ViT Epoch 24/25


100%|██████████| 266/266 [00:35<00:00,  7.52it/s]


Epoch 24/25, Time: 35.40s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000

ViT Epoch 25/25


100%|██████████| 266/266 [00:35<00:00,  7.54it/s]


Epoch 25/25, Time: 35.30s, Loss: nan, Macro F1: 0.0000, Val Loss: nan, Val Macro F1: 0.0000


# Compiling and Publishing Results

In [45]:
print("CNN EWC Subdiagnostic Classification Report:")
cnn_ewc_sub_report = evaluate_model(cnn_sub_model, X_test, y_test_sub, classes_sub)

print("CNN EWC Superdiagnostic Classification Report:")
cnn_ewc_super_report = evaluate_model(cnn_super_model, X_test, y_test_super, classes_super)

print("ResNet EWC Subdiagnostic Classification Report:")
resnet_ewc_sub_report = evaluate_model(resnet_sub_model, X_test, y_test_sub, classes_sub)

print("ResNet EWC Superdiagnostic Classification Report:")
resnet_ewc_super_report = evaluate_model(resnet_super_model, X_test, y_test_super, classes_super)

print("ViT EWC Subdiagnostic Classification Report:")
vit_ewc_sub_report = evaluate_model(vit_sub_model, X_test, y_test_sub, classes_sub)

print("ViT EWC Superdiagnostic Classification Report:")
vit_ewc_super_report = evaluate_model(vit_super_model, X_test, y_test_super, classes_super)


CNN EWC Subdiagnostic Classification Report:
68/68 [==============================] - 0s 716us/step
              precision    recall  f1-score   support

         AMI       0.00      0.00      0.00       306
       CLBBB       0.00      0.00      0.00        54
       CRBBB       0.00      0.00      0.00        54
       ILBBB       0.00      0.00      0.00         8
         IMI       0.00      0.00      0.00       327
       IRBBB       0.00      0.00      0.00       112
        ISCA       0.00      0.00      0.00        93
        ISCI       0.00      0.00      0.00        40
        ISC_       0.00      0.00      0.00       128
        IVCD       0.00      0.00      0.00        79
   LAFB/LPFB       0.00      0.00      0.00       179
     LAO/LAE       0.00      0.00      0.00        42
         LMI       0.00      0.00      0.00        20
         LVH       0.00      0.00      0.00       214
        NORM       0.00      0.00      0.00       963
        NST_       0.00      0.00  

In [46]:
print("CNN SI Subdiagnostic Classification Report:")
cnn_si_sub_report = evaluate_model(cnn_sub_model, X_test, y_test_sub, classes_sub)

print("CNN SI Superdiagnostic Classification Report:")
cnn_si_super_report = evaluate_model(cnn_super_model, X_test, y_test_super, classes_super)

print("ResNet SI Subdiagnostic Classification Report:")
resnet_si_sub_report = evaluate_model(resnet_sub_model, X_test, y_test_sub, classes_sub)

print("ResNet SI Superdiagnostic Classification Report:")
resnet_si_super_report = evaluate_model(resnet_super_model, X_test, y_test_super, classes_super)

print("ViT SI Subdiagnostic Classification Report:")
vit_si_sub_report = evaluate_model(vit_sub_model, X_test, y_test_sub, classes_sub)

print("ViT SI Superdiagnostic Classification Report:")
vit_si_super_report = evaluate_model(vit_super_model, X_test, y_test_super, classes_super)


CNN SI Subdiagnostic Classification Report:
68/68 [==============================] - 0s 886us/step
              precision    recall  f1-score   support

         AMI       0.00      0.00      0.00       306
       CLBBB       0.00      0.00      0.00        54
       CRBBB       0.00      0.00      0.00        54
       ILBBB       0.00      0.00      0.00         8
         IMI       0.00      0.00      0.00       327
       IRBBB       0.00      0.00      0.00       112
        ISCA       0.00      0.00      0.00        93
        ISCI       0.00      0.00      0.00        40
        ISC_       0.00      0.00      0.00       128
        IVCD       0.00      0.00      0.00        79
   LAFB/LPFB       0.00      0.00      0.00       179
     LAO/LAE       0.00      0.00      0.00        42
         LMI       0.00      0.00      0.00        20
         LVH       0.00      0.00      0.00       214
        NORM       0.00      0.00      0.00       963
        NST_       0.00      0.00   

In [47]:
def get_macro_f1(report_dict):
    return report_dict['macro avg']['f1-score']

results = {
    'Model': [],
    'Task': [],
    'Macro F1-score': []
}

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['Superdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_super_report),
    get_macro_f1(resnet_super_report),
    get_macro_f1(vit_super_report)
])

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['Subdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_sub_report),
    get_macro_f1(resnet_sub_report),
    get_macro_f1(vit_sub_report)
])

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['EWC Subdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_ewc_sub_report),
    get_macro_f1(resnet_ewc_sub_report),
    get_macro_f1(vit_ewc_sub_report)
])

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['EWC Superdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_ewc_super_report),
    get_macro_f1(resnet_ewc_super_report),
    get_macro_f1(vit_ewc_super_report)
])

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['SI Subdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_si_sub_report),
    get_macro_f1(resnet_si_sub_report),
    get_macro_f1(vit_si_sub_report)
])

results['Model'].extend(['CNN', 'ResNet', 'ViT'])
results['Task'].extend(['SI Superdiagnostic'] * 3)
results['Macro F1-score'].extend([
    get_macro_f1(cnn_si_super_report),
    get_macro_f1(resnet_si_super_report),
    get_macro_f1(vit_si_super_report)
])

results_df = pd.DataFrame(results)
print("\nSummary of Classification Performance:")
print(results_df)



Summary of Classification Performance:
     Model                 Task  Macro F1-score
0      CNN      Superdiagnostic        0.732153
1   ResNet      Superdiagnostic        0.734195
2      ViT      Superdiagnostic        0.673153
3      CNN        Subdiagnostic        0.419499
4   ResNet        Subdiagnostic        0.400487
5      ViT        Subdiagnostic        0.279755
6      CNN    EWC Subdiagnostic        0.000000
7   ResNet    EWC Subdiagnostic        0.000000
8      ViT    EWC Subdiagnostic        0.000000
9      CNN  EWC Superdiagnostic        0.000000
10  ResNet  EWC Superdiagnostic        0.000000
11     ViT  EWC Superdiagnostic        0.000000
12     CNN     SI Subdiagnostic        0.000000
13  ResNet     SI Subdiagnostic        0.000000
14     ViT     SI Subdiagnostic        0.000000
15     CNN   SI Superdiagnostic        0.000000
16  ResNet   SI Superdiagnostic        0.000000
17     ViT   SI Superdiagnostic        0.000000


# GPT Driven code for Federated Learning

In [48]:
# ----------------------------------------
# Federated Learning with Continual Learning (EWC & SI)
# ----------------------------------------

print("Starting Federated Learning with Continual Learning (EWC & SI)")

# Number of clients to simulate
num_clients = 5

# Communication rounds
communication_rounds = 5

# Local training epochs per client
local_epochs = 1

# Batch size for local training
local_batch_size = 64

# Lambda values for EWC and SI
lambda_ewc = 1000
lambda_si = 1.0

# Function to split data among clients
def split_data(X, y_super, y_sub, num_clients):
    client_data = []
    data_per_client = len(X) // num_clients
    for i in range(num_clients):
        start_idx = i * data_per_client
        end_idx = (i + 1) * data_per_client if i < num_clients - 1 else len(X)
        X_client = X[start_idx:end_idx]
        y_client_super = y_super[start_idx:end_idx]
        y_client_sub = y_sub[start_idx:end_idx]
        client_data.append((X_client, y_client_super, y_client_sub))
    return client_data

# Split the training data among clients
client_data = split_data(X_train, y_train_super, y_train_sub, num_clients)

# Function to clone a model and set weights
def clone_model_weights(model):
    cloned_model = tf.keras.models.clone_model(model)
    cloned_model.set_weights(model.get_weights())
    return cloned_model

# Function to average client weights
def average_weights(client_weights):
    avg_weights = []
    for weights in zip(*client_weights):
        avg = np.mean(weights, axis=0)
        avg_weights.append(avg)
    return avg_weights

# Define Federated Learning with EWC and SI
def federated_training(model_type, create_model_fn, input_shape, num_classes_super, num_classes_sub, classes_super, classes_sub):
    print(f"--- Federated Training for {model_type} ---")
    
    # Initialize the global model
    global_model_super = create_model_fn(input_shape, num_classes_super)
    global_weights_super = global_model_super.get_weights()
    
    # Initialize the global models for subdiagnostic tasks
    global_model_sub = modify_model_for_subdiagnostic(global_model_super, num_classes_sub)
    global_weights_sub = global_model_sub.get_weights()
    
    for round_num in range(communication_rounds):
        print(f"Communication Round {round_num+1}/{communication_rounds}")
        client_weights_super = []
        client_weights_sub = []
        
        for client_idx, (X_client, y_client_super, y_client_sub) in enumerate(client_data):
            print(f" - Client {client_idx+1}/{num_clients} local training")
            
            # Clone global models
            client_model_super = clone_model_weights(global_model_super)
            client_model_sub = clone_model_weights(global_model_sub)
            
            # --------------------
            # Local Training on Superdiagnostic Task
            # --------------------
            client_model_super.compile(
                optimizer='adam',
                loss='binary_crossentropy',
                metrics=[macro_f1]
            )
            client_model_super.fit(
                X_client, y_client_super,
                epochs=local_epochs,
                batch_size=local_batch_size,
                verbose=0
            )
            
            # Initialize EWC for the client
            exclude_params_super = [id(w) for w in client_model_super.layers[-1].trainable_weights]
            ewc_client = EWC(client_model_super, X_client, y_client_super, exclude_params=exclude_params_super)
            
            # --------------------
            # Local Training on Subdiagnostic Task with EWC
            # --------------------
            client_model_sub = modify_model_for_subdiagnostic(client_model_super, num_classes_sub)
            
            # Define EWC loss for subdiagnostic task
            def ewc_loss_sub(y_true, y_pred):
                task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
                ewc_penalty = ewc_client.penalty(client_model_sub)
                total_loss = task_loss + (lambda_ewc / 2) * ewc_penalty
                return total_loss
            
            client_model_sub.compile(
                optimizer='adam',
                loss=ewc_loss_sub,
                metrics=[macro_f1]
            )
            client_model_sub.fit(
                X_client, y_client_sub,
                epochs=local_epochs,
                batch_size=local_batch_size,
                verbose=0
            )
            
            # Initialize SI for the client
            exclude_params_sub = [id(w) for w in client_model_sub.layers[-1].trainable_weights]
            si_client = SI(client_model_sub, damping_factor=0.1, exclude_params=exclude_params_sub)
            
            # --------------------
            # Local Training on Subdiagnostic Task with SI
            # --------------------
            # Define SI loss for subdiagnostic task
            def si_loss_sub(y_true, y_pred):
                task_loss = tf.keras.losses.binary_crossentropy(y_true, y_pred)
                si_penalty = si_client.penalty(client_model_sub)
                total_loss = task_loss + (lambda_si / 2) * si_penalty
                return total_loss
            
            client_model_sub.compile(
                optimizer='adam',
                loss=si_loss_sub,
                metrics=[macro_f1]
            )
            client_model_sub.fit(
                X_client, y_client_sub,
                epochs=local_epochs,
                batch_size=local_batch_size,
                verbose=0
            )
            
            # Update EWC and SI
            # For EWC, we've already computed the penalty during training
            # For SI, update omega after training
            si_client.update_omega(client_model_sub)
            
            # Collect client weights
            client_weights_super.append(client_model_super.get_weights())
            client_weights_sub.append(client_model_sub.get_weights())
        
        # Aggregate client weights to update global models
        global_weights_super = average_weights(client_weights_super)
        global_weights_sub = average_weights(client_weights_sub)
        
        # Set the aggregated weights to global models
        global_model_super.set_weights(global_weights_super)
        global_model_sub.set_weights(global_weights_sub)
    
    # After federated training, evaluate the global models
    print(f"--- Evaluation for {model_type} after Federated Learning ---")
    
    # Evaluate on Superdiagnostic Task
    print(f"Evaluating Global {model_type} Model on Superdiagnostic Task:")
    super_report = evaluate_model(global_model_super, X_test, y_test_super, classes_super)
    
    # Evaluate on Subdiagnostic Task
    print(f"Evaluating Global {model_type} Model on Subdiagnostic Task:")
    global_model_sub = modify_model_for_subdiagnostic(global_model_super, num_classes_sub)
    sub_report = evaluate_model(global_model_sub, X_test, y_test_sub, classes_sub)
    
    return super_report, sub_report

# ----------------------------------------
# Start Federated Training for CNN, ResNet, ViT
# ----------------------------------------

# Initialize a dictionary to store federated reports
federated_reports = {
    'Model': [],
    'Task': [],
    'Macro F1-score': []
}

# List of models to federate
models_to_federate = [
    ('CNN', create_cnn_model),
    ('ResNet', create_resnet_model),
    ('ViT', create_vit_model)
]

for model_name, create_model_fn in models_to_federate:
    # Determine number of classes based on model type
    if model_name in ['CNN', 'ResNet', 'ViT']:
        num_classes_sub = y_train_sub.shape[1]
    
    # Federated training
    super_report, sub_report = federated_training(
        model_type=model_name,
        create_model_fn=create_model_fn,
        input_shape=input_shape,
        num_classes_super=num_classes_super,
        num_classes_sub=num_classes_sub,
        classes_super=classes_super,
        classes_sub=classes_sub
    )
    
    # Store the reports
    federated_reports['Model'].extend([model_name, model_name])
    federated_reports['Task'].extend(['Federated Superdiagnostic', 'Federated Subdiagnostic'])
    federated_reports['Macro F1-score'].extend([
        get_macro_f1(super_report),
        get_macro_f1(sub_report)
    ])

# ----------------------------------------
# Update and Publish the Summary Table with Federated Results
# ----------------------------------------

# Convert federated reports to DataFrame
federated_df = pd.DataFrame(federated_reports)

# Append federated results to the existing summary table
results = results.append(federated_df, ignore_index=True)

print("Updated Summary of Classification Performance with Federated Learning:")
print(results)


Starting Federated Learning with Continual Learning (EWC & SI)

--- Federated Training for CNN ---

Communication Round 1/5
 - Client 1/5 local training
 - Client 2/5 local training
 - Client 3/5 local training
 - Client 4/5 local training
 - Client 5/5 local training

Communication Round 2/5
 - Client 1/5 local training
 - Client 2/5 local training
 - Client 3/5 local training
 - Client 4/5 local training
 - Client 5/5 local training

Communication Round 3/5
 - Client 1/5 local training
 - Client 2/5 local training
 - Client 3/5 local training
 - Client 4/5 local training
 - Client 5/5 local training

Communication Round 4/5
 - Client 1/5 local training
 - Client 2/5 local training
 - Client 3/5 local training
 - Client 4/5 local training
 - Client 5/5 local training

Communication Round 5/5
 - Client 1/5 local training
 - Client 2/5 local training
 - Client 3/5 local training
 - Client 4/5 local training
 - Client 5/5 local training

--- Evaluation for CNN after Federated Learning -

KeyboardInterrupt: 